In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import csv
import time
from os.path import isfile
import pandas as pd
import re
import spacy
import matplotlib.pyplot as plt
import Tools.processing as proc

model = SentenceTransformer('all-mpnet-base-v2')

In [3]:


titles_df = ["Text", "Score"]
df = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/Positive_Scores/sentiment labelled sentences/amazon_cells_labelled.txt", names=titles_df, sep='\t')
df2 = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/Positive_Scores/sentiment labelled sentences/imdb_labelled.txt", names=titles_df, sep='\t')
df3 = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/Positive_Scores/sentiment labelled sentences/yelp_labelled.txt", names=titles_df, sep='\t')

In [4]:
encodings = df["Text"].apply(lambda x: model.encode(x))
encodings2 = df2["Text"].apply(lambda x: model.encode(x))
encodings3 = df3["Text"].apply(lambda x: model.encode(x))

In [5]:
few_shot = pd.read_json("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_pooled_embeddings.json")
positive = [float(text) for text in few_shot["positive"]]
negative = [float(text) for text in few_shot["negative"]]

In [68]:
def compare_embeddings(embed, positive=few_shot['positive'], negative=few_shot['negative'], hyper=0.2, visible=False):
    embed_temp = np.array([float(x) for x in embed], dtype=np.float32)
    positive = np.array([float(x) for x in positive], dtype=np.float32)
    negative = np.array([float(x) for x in negative], dtype=np.float32)

    similarity_pos = util.cos_sim(embed_temp, positive)[0][0]
    similarity_neg = util.cos_sim(embed_temp, negative)[0][0]
    #print(f"{similarity_pos} : {similarity_neg} : {similarity_pos >= similarity_neg}")
    diff = similarity_pos - (similarity_neg + (similarity_pos *hyper))
    if visible:
        print(f"Difference in embedding {diff}")
    return (diff >= 0).item()
#df['Encodings'] = [float(x) for x in df["Encodings"]]
#df["Test_Score"] = df['Encodings'].apply(lambda x: compare_embeddings(x))

#encodings = encodings.astype(np.float32)
df["Train_Score"] = encodings.apply(lambda x: compare_embeddings(x))
df2["Train_Score"] = encodings2.apply(lambda x: compare_embeddings(x))
df3["Train_Score"] = encodings3.apply(lambda x: compare_embeddings(x))

In [93]:
train_scores = df["Train_Score"]
test_scores = df["Score"]

def build_confusion_matrix(train_scores, test_scores):
    matrix = np.zeros((2, 2))
    for train, test in zip(train_scores, test_scores):
        train_temp = 1 if train else 0
        test_temp = 1 if test else 0
        #print(f"Train: {train}, Test: {test}")
        matrix[test_temp][train_temp] += 1
    return matrix

def find_test_score(train_score, test_score):
    count = 0
    for test, train in zip(test_score, train_score):
        count += 1 if test == train else 0
    return count/len(test_score)

def print_test_log(df : pd.DataFrame):
    print(build_confusion_matrix(df["Train_Score"], df["Label"]))
    score = find_test_score(df["Train_Score"], df["Label"])
    print(f"Test Accuarcy: {score}")

# print("Amazon")
# print_test_log(df)
# print()
# print("IMBD")
# print_test_log(df2)
# print()
# print("Yelp")
# print_test_log(df3)

In [90]:
df_standford = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSentences.txt", sep='\t')
df_standford['Label'] = pd.read_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Lab_Tests/Data_Storage/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSplit.txt")['splitset_label']

df_standford["Encodings"] = df_standford["sentence"].apply(lambda x: model.encode(x))
df_standford.head()

,sentence_index,sentence,Label,Encodings
0,1,The Rock is destined to be the 21st Century 's...,1,"[0.010585851, -0.04431008, 0.0058608465, -0.03..."
1,2,The gorgeously elaborate continuation of `` Th...,1,"[0.040952645, 0.002876757, -0.027633326, 0.023..."
2,3,Effective but too-tepid biopic,2,"[0.040901687, 0.110521756, 0.02460098, -0.0006..."
3,4,If you sometimes like to go to the movies to h...,2,"[-0.0031407902, 0.03039741, -0.018152902, -0.0..."
4,5,"Emerges as something rare , an issue movie tha...",2,"[0.009049049, 0.03680868, 0.029847197, 0.01476..."


In [103]:
#df_standford["Train_Score"] = df_standford["Encodings"].apply(lambda x: compare_embeddings(x))
#df_standford["Label"] = [label == 2 for label in df_standford["Label"]]


#matrix = build_confusion_matrix(df_standford["Train_Score"], df_standford["Label"])
#print(matrix)
hyper = np.arange(2, 4, 0.1)
# for hyp in hyper:
#     df_standford["Train_Score"] = df_standford["Encodings"].apply(lambda x: compare_embeddings(x, hyper=hyp))
#     score = find_test_score(df_standford["Train_Score"], df_standford["Label"])
#     print(f"{hyp}: {score}")
hyper_param = 3.8

df_standford["Train_Score"] = df_standford["Encodings"].apply(lambda x: compare_embeddings(x, hyper=hyper_param))
score = find_test_score(df_standford["Train_Score"], df_standford["Label"])
print(f"Test Score: {score}")

Test Score: 0.5757908055672712
